# Downloading data from FEWS NET USGS

In [1]:
# imports
%load_ext jupyter_black
from pathlib import Path
import os
import requests, zipfile, io
from bs4 import BeautifulSoup

In [4]:
data_dir = Path(os.getenv("HAM_DIR"))

## NDVI

In [5]:
ndvi_dir = data_dir / "vegetation/ndvi/"

In [14]:
URL_ndvi = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/fews/web/africa/west/pentadal/eviirs/ndvi/temporallysmoothedndvi/downloads/pentadal/"


def list_files(URL, ext):
    page = requests.get(URL, verify=False).text
    soup = BeautifulSoup(page, "html.parser")
    for node in soup.find_all("a"):
        if node.get("href").endswith(ext):
            file_url = [
                URL + "/" + node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
            file_list = [
                node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
    return file_list


ndvi_file_list = list_files(URL=URL_ndvi, ext="zip")

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
for file in ndvi_file_list:
    if file in ndvi_file_list:
        req = requests.get(URL_ndvi + file, verify=False, stream=True)
        with zipfile.ZipFile(io.BytesIO(req.content)) as zObj:
            fileNames = zObj.namelist()
            for fileName in fileNames:
                if fileName.endswith("tif"):
                    content = zObj.open(fileName).read()
                    open(ndvi_dir / fileName, "wb").write(content)

## WRSI

In [ ]:
wrsicroplands_dir = data_dir / "water_requirements/wrsi/croplands/"
wrsirangelands_dir = data_dir / "water_requirements/wrsi/rangelands/"

URL_wrsi = ""
wrsi_file_list = list_files(URL=, ext="zip")